In [1]:

'''
不同训练方式的区别

训练方法    计算误差的答案   反馈的计算方式

sft        由人类生成         数学公式计算分数（交叉熵）
rlhf       无              人类或模型打分(1-10)

RL         无              游戏规则打分
模型蒸馏    由模型生成        数学公式计算分数（交叉熵）
DPO        无              人类打分(1 or 0)
PPO        无              游戏规则打分（梯度打分）
'''

'''r1 训练流程

————————————————1 deepseek v3 作为一个基座模型 根据输入生成答案
训练部分
1 训练输出格式[sft] -> 为了通过代码批量识别输出答案的位置。（可以跳过，但会花很长时间摸索输出格式，在这之前都是无效训练）
2 训练结果正确性[RL] -> 提高正确率。

打分部分
打分 = 格式分 + 正确性分

根据打分结果，去更新参数，训练出 r1-zero

————————————————2 使用 deepseek r1-zero 根据 大量 输入 生成 大量 答案
cot连续推理数据 = deepseek r1-zero 生成 [60万条]
非推理数据 = deepseek v3 生成 [20万条]

使用这些样本去训练出 r1

————————————————3 得到模型 deepseek r1
'''

''' 复现成本

基座模型:qwen2.5-0.5b  约1.1g大小
推理数据集: openai/gsm8k (全数学题)
训练配置:8000条数据 累积4次更新1次 = 约2000 step
显卡类型:h20 96g
显存占用:23.430g
训练时间:5h
开始有效学习步数:92(奖励值≠0)


基座模型:qwen2.5-1.5b 约3.4g大小
推理数据集: openai/gsm8k (全数学题)
训练配置:8000条数据 累积4次更新1次 = 约2000 step
显卡类型:h20 96g
显存占用:43.434g
训练时间:6.5h
开始有效学习步数:63(奖励值≠0)

'''

' 复现成本\n\n基座模型 qwen 0.5b\n推理数据集 openai/gsm8k (全数学题)\n8000 样本 累积 4 个样本 更新 1 次 = 2000 step\n\na100 40g\n27g显存\n4-5 小时\n'